# The Puppeteer, a fill-in-the-blanks radio show

runtime: 10–15 minutes

#### set up the grid

In [1]:
import ipywidgets
ipywidgets.__version__

'8.0.6'

In [2]:
import os
from puppeteer import Show, ShowConfig, Broca
import pickle
import random

broca = Broca('interface', save_to_file=False)
broca.init_voices()

scenarios = ['']
for fname in os.listdir('scenarios'):
    if '.pickle' in fname:
        scenarios.append(fname.replace('.pickle', ''))
scenarios = sorted(scenarios)
voices = [a['name'] for a in Broca.STOCK_AGENTS]

In [3]:
from ipywidgets import Button, GridBox, Layout, ButtonStyle, interact, interact_manual, widgets
textarea_layout = widgets.Layout(width='500px')
textarea_layout_big = widgets.Layout(width='600px')
text_layout = widgets.Layout(width='300px', description_width='150px')
text_layout_big = widgets.Layout(width='375px', description_width='150px')
textarea_rows = 12

show_name_w = widgets.Text(placeholder='Ali Baba', description='Name:', disabled=False)
secret_knowledge_w = widgets.Text(placeholder='"open sesame"', description='info:', disabled=False, layout=text_layout_big)
secret_knowledge_name_w = widgets.Text(placeholder='magical phrase', description='info name:')
use_gpt4_w = widgets.Checkbox(value=False, description='use gpt-4', disabled=False)

target_name_w = widgets.Text(placeholder="Ali Baba", description='name:', disabled=False)

# drop-down menus
target_voice_w = widgets.Dropdown(options=voices, value=random.choice(voices), description='Ch voice', disabled=False)
load_selector_w = widgets.Dropdown(options=scenarios, value='', description='', disabled=False)
rating_w = widgets.Dropdown(options=['G', 'PG', 'PG-13', 'R'], value='PG-13', description='Rating', disabled=True)

# radio buttons?
scenes_w = widgets.Text(placeholder='2 (2 scenes ~= 12 minutes)', description='Scenes:', disabled=False)

# prior info for main characters
target_system_prompt_w = widgets.Textarea(rows=textarea_rows, layout=textarea_layout_big, description='Ch prompt', disabled=False,
    placeholder='''Instructions to the main character about who they are. Example:

You are a poor woodcutter and an honest person who discovers the secret of a thieves' den, the magic phrase "open sesame".

The instructions should include the info and the name of the info. And narratively, it's good to make it clear the character shouldn't divulge the info without making it impossible either. Make the character want something.
''',)
puppeteer_knowledge_w = widgets.Textarea(rows=textarea_rows, layout=textarea_layout_big, placeholder='''What the puppeteer knows. Example: 

Our target is a poor woodcutter in the Islamic Golden Age.''', description='Dossier:', disabled=False)

# scenes
scene_1_w = widgets.Textarea(rows=textarea_rows, layout=textarea_layout, description='1:', disabled=False, placeholder='''The beginning of scene 1. Example:

INTERIOR – MORNING.
Ali Baba sits in his small wood-carving studio, thinking about the magical phrase he overheard when he was cutting firewood in the forest, and saw the group of 40 thieves visiting their stored treasure.
''')
scene_2_w = widgets.Textarea(rows=textarea_rows, layout=textarea_layout, description='2:', disabled=False, placeholder='''The beginning of scene 2. Can be blank.
''')
scene_3_w = widgets.Textarea(rows=textarea_rows, layout=textarea_layout, description='3:', disabled=False, placeholder='''The beginning of scene 3. Can be blank.
''')

In [4]:
# Define the target function that receives the widget value
def play(cfg):
    show = Show(cfg, save_to_file=False)
    show.replay()
    
def record(cfg):
    print('use gpt4?', use_gpt4_w.value)
    show = Show(cfg, save_to_file=True, spoken=False, quick_and_dirty=not use_gpt4_w.value)
    broca.say_this("Recording...")
    try:
        show.intro()
        show.run()
        show.end()
    except Exception as e:
        broca.say_this(f"Recording failed. Reason: {e}")
        raise(e)
    broca.say_this("Recording finished")

def load_config(spoken):
    show_name = show_name_w.value
    secret_knowledge = secret_knowledge_w.value
    secret_knowledge_name = secret_knowledge_name_w.value
    target_name = target_name_w.value
    target_voice = target_voice_w.value
    scene_count = scenes_w.value if scenes_w.value else 2
    target_system_prompt = target_system_prompt_w.value
    scene_1 = scene_1_w.value
    scene_2 = scene_2_w.value
    scene_3 = scene_3_w.value
    scenes = [scene_1, scene_2, scene_3]
    puppeteer_knowledge = puppeteer_knowledge_w.value

    cfg = ShowConfig(name=show_name, secret_knowledge=secret_knowledge, 
        secret_knowledge_name=secret_knowledge_name, target_name=target_name,
        target_voice=target_voice, target_system_prompt=target_system_prompt, 
        puppeteer_knowledge=puppeteer_knowledge,
        rounds=scene_count, scenes=scenes, spoken=True)
    return cfg

def populate_fields(cfg):
    show_name_w.value = cfg.name
    secret_knowledge_w.value = cfg.secret_knowledge
    secret_knowledge_name_w.value = cfg.secret_knowledge_name
    target_name_w.value = cfg.target_name
    target_voice_w.value = cfg.target_voice
    target_system_prompt_w.value = cfg.target_system_prompt
    scene_1_w.value = cfg.scenes[0]
    if len(cfg.scenes) > 1:
        scene_2_w.value = cfg.scenes[1]
    if len(cfg.scenes) > 2:
        scene_3_w.value = cfg.scenes[2]
    scenes_w.value = str(cfg.rounds)
    puppeteer_knowledge_w.value = cfg.puppeteer_knowledge

def on_start_button_clicked(button):
    cfg = load_config(spoken=True)
    play(cfg)

def on_prerecord_button_clicked(button):
    cfg = load_config(spoken=False)
    record(cfg)

global loading
loading = False
def on_load_button_clicked(button):
    show_name = load_selector_w.value
    cfg = ShowConfig.load(show_name)
    global loading
    loading = True
    populate_fields(cfg)
    loading = False

def on_save_button_clicked(button):
    cfg = load_config(spoken=False)
    overwrite_okay = overwrite_okay_w.value
    cfg.save(overwrite_okay)
    scenarios.append(cfg.name)
    load_selector_w.options = sorted(list(set(scenarios)))
    broca.say_this(cfg.name + ' saved.')

# Create a Button widget for submitting the input
start_button = widgets.Button(description='Play')
# Bind the callback function to the button's click event
start_button.on_click(on_start_button_clicked)

# Create a Button widget for submitting the input
prerecord_button = widgets.Button(description='Pre-record')
# Bind the callback function to the button's click event
prerecord_button.on_click(on_prerecord_button_clicked)

# Create a Button widget for submitting the input
load_button = widgets.Button(description='Load')
# Bind the callback function to the button's click event
load_button.on_click(on_load_button_clicked)

# Create a Button widget for submitting the input
save_button = widgets.Button(description='Save')
# Bind the callback function to the button's click event
save_button.on_click(on_save_button_clicked)

overwrite_okay_w = widgets.Checkbox(value=False, description='overwrite ok', disabled=False)

In [5]:
def say_something(button):
    voice = target_voice_w.value
    dialogue = 'My name is ' + voice
    if target_name_w.value != '':
        dialogue = 'My name is ' + target_name_w.value
    if show_name_w.value != '':
        dialogue += '... Welcome to ' + show_name_w.value
    broca.say_this(dialogue, voice_override=voice)
    
speak_button = widgets.Button(description='Speak')
speak_button.on_click(say_something)

In [6]:
show_grid = widgets.GridBox([show_name_w, secret_knowledge_name_w, secret_knowledge_w, use_gpt4_w], layout=widgets.Layout(
    grid_template_columns='repeat(4, 300px)'
))
char_grid = widgets.GridBox([target_name_w, target_voice_w, speak_button, rating_w], layout=widgets.Layout(
    grid_template_columns='repeat(4, 300px)'
))
txtb_grid = widgets.GridBox([target_system_prompt_w, puppeteer_knowledge_w], layout=widgets.Layout(
    grid_template_columns='repeat(3, 650px)'
))
scene_grid = widgets.GridBox([scene_1_w, scene_2_w, scene_3_w], layout=widgets.Layout(
    grid_template_columns='repeat(3, 450px)'
))
start_grid = widgets.GridBox([scenes_w, load_selector_w, load_button, save_button, prerecord_button, start_button, overwrite_okay_w], layout=widgets.Layout(
    grid_template_columns='repeat(4, 250px)'
))

# Display the grid
# display(grid)
# Create a Label widget for the row label
row1_label = widgets.Label(value='The Show: name and the secret information', layout=widgets.Layout(width='300px'))
row2_label = widgets.Label(value='Character', layout=widgets.Layout(width='300px'))
row3_label = widgets.Label(value='Secret information', layout=widgets.Layout(width='300px'))
row4_label = widgets.Label(value='Scenes (only the first is necessary)', layout=widgets.Layout(width='300px'))

# Create a VBox to stack the row label and the grid vertically
vbox = widgets.VBox([row1_label, show_grid, row2_label, char_grid, row3_label, txtb_grid, row4_label, scene_grid, start_grid])

#### display the grid

In [7]:
# Display the VBox
display(vbox)

todo

- find ways of constraining length (just use rounds)
- main character voice selection
- possible to play audio file over ads? [cmd](https://www.macworld.com/article/194771/105audioterm.html)
- factor out the rest of the prompts

widget layout
- target design (name, voice, description with starter text with the target's name etc in it). should all be text fields, except voice selection.
- passphrase and passphrase type. 
- desired length (experimental). 
- puppeteer knowledge. 
- scene design (fill out at least the first scene)

In [8]:
# cfg = load_config(False)

In [9]:
1/0

ZeroDivisionError: division by zero

In [ ]:
def get_agent(new_agent_text):
    try:
        agent_name, next_agent = new_agent_text.split('>>')
    except:
        reformatted_agent_text = query(f'''The following contains a description of one of our Agents, including their name. 
{new_agent_text}
Please format the text so that it is in the format: 
AGENT NAME >> Rest of text.

Do not otherwise change the text.''')
        try:
            agent_name, next_agent = reformatted_agent_text.split('>>')
        except:
            lines = reformatted_agent_text.strip().split('\n')
            agent_name = lines[0].strip()
            if len(agent_name) == 0:
                agent_name = self.cfg.default_agent_name
            next_agent = '\n'.join(lines[1:])
    try:
        agent_name = agent_name.split(': ')[1].strip()
    except:
        print('failed to get name from agent spec:\n' + new_agent_text)
        agent_name = self.cfg.default_agent_name # just use the default
    return agent_name.upper(), next_agent
